<a href="https://colab.research.google.com/github/IJLee0812/Bitcoin-Auto-Trading-Project-Using-Volatility-Breakthrough-Strategy/blob/main/Python_%EB%B3%80%EB%8F%99%EC%84%B1_%EB%8F%8C%ED%8C%8C_%EC%A0%84%EB%9E%B5%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_BTC_%EC%9E%90%EB%8F%99%EB%A7%A4%EB%A7%A4_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 변동성 돌파 전략을 이용한 비트코인 매매 프로젝트

▶ 비트코인 상승장을 타겟팅한 변동성 돌파 전략을 구현 후 상승장이 다가오면 사용해 보고자 함.

⚠️ 해당 소스코드 로직 실사용에 따른 제 책임은 없습니다.

# 1. 필요 라이브러리 install 및 import

In [ ]:
!pip install pyupbit
import pyupbit
import datetime
import time, calendar

# 2. 함수 정의

In [ ]:
"""
    get_targetPrice 함수 : 변동성 돌파 전략을 사용하기 위한 타깃 가격 구하는 함수.
    df는 pyupbit의 get_ohlcv() 함수로 받아온 차트 데이터(메인함수에서), 최신의 데이터가 마지막에 저장되어 있으므로
    인덱스 -1은 오늘, -2는 어제의 데이터를 나타낸다. 전일 고가와 저가의 차이를 구하여 Range를 구하고, 당일 시가에
    range * K 값을 더하여 targetPrice를 구하여 리턴한다.
"""
def get_targetPrice(df, K):
    range = df['high'][-2] - df['low'][-2]
    return df['open'][-1] + range * K

"""
    buy_all 함수 : 현재 가지고 있는 KRW 잔액 모두를 사용하여 시장가로 BTC 매수.
    (업비트 거래소는 최소 주문 금액 5,000원이므로 이를 만족할 때만 주문이 실행하도록 조건문을 설정함.)
"""
def buy_all(coin):
    balance = upbit.get_balance("KRW")
    if balance > 5000:
        upbit.buy_market_order(coin, balance)
"""
    sell_all 함수 : BTC를 전량 시장가 매도.
    (업비트 거래소는 최소 주문 금액 5,000원이므로 이를 만족할 때만 주문이 실행하도록 조건문을 설정함.)
"""
def sell_all(coin): # 가지고 있는 특정 가상화폐를 전량 시장가 매도.
    balance = upbit.get_balance(coin)
    price = pyupbit.get_current_price(coin)
    if price * balance > 5000:
        upbit.sell_market_order(coin, balance)

# 3. 메인함수 작성

In [ ]:
if __name__ == '__main__':
    try:
        upbit = pyupbit.Upbit('access_key', 'secret_key')   # 개인의 업비트 (Access Key, Secret Key) 기입

        coin = "KRW-BTC"    # 매매를 진행할 가상화폐 Ticker
        fees = 0.0005       # 수수료(0.05%임에 유의)
        K = 0.5             # target price를 구할 때 사용할 K값 (백테스팅 결과, BTC는 K = 0.5가 적합)

        df = pyupbit.get_ohlcv(coin, count = 2, interval = "day")   # 시작 지점에서 차트를 불러옴.
        targetPrice = get_targetPrice(df, K)                        # 첫 targetPrice 설정

        """
            반복문 loop : 현재 가상화폐의 가격이 targetPrice보다 높아졌는지 체크하고, 이를 만족하면 전액 매수한다.
            매일 09:00 AM은 새로운 일봉이 시작되는 시간이다. 따라서 가지고 있는 BTC 전량 매도 후,
            차트 데이터와 targetPrice를 Update한다.
            BTC의 현재가가 targetPrice를 돌파해서 매수가 진행되고 나면, 다음날 시가에 이를 매도하기 전까지
            아무런 작업을 할 필요가 없기 때문에, 이 시간 동안은 프로그램을 쉬도록 한다.
        """
        while True:
            now = datetime.datetime.now()
            if now.hour == 9 and now.minute == 0 and now.second == 0:
                sell_all(coin)
                df = pyupbit.get_ohlcv(coin, count = 2, interval = "day")
                targetPrice = get_targetPrice(df, K)
            if targetPrice <= pyupbit.get_current_price(coin):
                buy_all(coin)
                start_time = df.index[-1]                               # 오늘 장 시작 시각의 시간 데이터 저장
                end_time = start_time + datetime.timedelta(days = 1)    # 다음 날 09:00 AM 값
                time.sleep((end_time - now).seconds)                    # 다음 날 장 시작시간까지 남은 시간이 나오고, 이를 초로 환산하여 다음날 장 시작까지 프로그램 멈춤
            time.sleep(1)
    except Exception as e:  # 예외처리
        print(e)
        time.sleep(1)